# Image class creation

## Class
We are going to use the class "ImageFolder"

In [2]:
import os

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Resize, Compose

In [3]:
data_dir = '/home/gines/Escritorio/TFG/data'
classes = os.listdir(data_dir)
print(classes)

['cartoon3D', 'real', 'cartoon2D']


The functions have different sizes, we must resize them to the same size, the images have a size of 
* Cartoon2D : 3840 width x 1634 height
* Cartoon3D : 1920 width x 816 height
* Real: 1920 wide x 816 high

So we choose the smallest: 1920 wide x 816 high

In [4]:
transform_custom = Compose(
    [Resize((1920, 816)),
     ToTensor()])

dataset = ImageFolder(data_dir, transform = transform_custom)

## Training and validation dataset
We generate training and validation
from torch.utils.data import random_split dataset

In [5]:
import torch

from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader

In [6]:
random_seed = 42
torch.manual_seed(random_seed);

val_size = 10
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(155, 10)

In [7]:
batch_size=12

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

# Modelo creation

## Importación de datos

## Base class
We define "ImageClassificationBase" class to use it later.

In [8]:
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

## Modelo StarWars

In [10]:
class StarWarsCnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, xb):
        return self.network(xb)

model = StarWarsCnnModel()

In [ ]:
for images, labels in train_dl:
    print('images.shape:', images.shape)
    out = model(images)
    print('out.shape:', out.shape)
    print('out[0]:', out[0])
    break

images.shape: torch.Size([12, 3, 1920, 816])


ERROR

Traceback (most recent call last):

  File "/home/gines/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  
  File "/home/gines/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  
  File "/home/gines/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  
    File "/home/gines/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 844, in launch_instance
    app.initialize(argv)
  
    File "/home/gines/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 87, in inner
    return method(app, *args, **kwargs)
 
    File "/home/gines/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 567, in initialize
    self.init_sockets()
 
    File "/home/gines/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 271, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  
    File "/home/gines/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 218, in _bind_socket
    return self._try_bind_socket(s, port)
  
    File "/home/gines/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 194, in _try_bind_socket
    s.bind("tcp://%s:%i" % (self.ip, port))
 
    File "zmq/backend/cython/socket.pyx", line 550, in zmq.backend.cython.socket.Socket.bind
 
    File "zmq/backend/cython/checkrc.pxd", line 26, in zmq.backend.cython.checkrc._check_rc
zmq.error.ZMQError: Address already in use
[I 10:01:25.013 LabApp] KernelRestarter: restarting kernel (2/5), keep random ports
kernel ce451f1c-41d0-4283-ba16-19896d932505 restarted


## Training using GPU

### Auxiliary functions

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

### Training

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return train_losses

Move data to memory

In [ ]:
device = get_default_device()
train_dl = DeviceDataLoader(train_dl, device) # podemos poner shuffle = True
#val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device);

In [ ]:
num_epochs = 2
lr = 0.001
opt_func = torch.optim.Adam

history = fit(num_epochs, lr, model, train_dl, train_dl, opt_func)
history